In [1]:
import numpy as np
import time


index = np.load("/home/jovyan/Sample_Based_Extension/WUSTL/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9998827239867143,
        0.9600649800293866, array([1, 1, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9996984331086939,
        0.9305545272093338, array([3, 3, 3, ..., 3, 3, 3])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.9984796002563319,
        0.7518383136072314, array([5, 5, 5, ..., 5, 5, 5])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9998701586995765,
        0.939002641264322, array([1, 1, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9971518682487759,
        0.8894642081911899, array([3, 3, 3, ..., 3, 3, 3])],
       ['FGSM', 0.3, 'Def

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1:
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [5]:
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import entropy
import numpy as np

def stratified_random_sample(X, y, fraction=None, count=None, random_state=42):
    assert (fraction is not None) ^ (count is not None), "Specify either fraction or count"

    rng = np.random.RandomState(seed=random_state)
    X_out, y_out = [], []

    for label in np.unique(y):
        X_c = X[y == label]
        y_c = y[y == label]

        n_total = len(X_c)
        n_sample = int(n_total * fraction) if fraction is not None else min(count, n_total)
        if n_sample == 0:
            n_sample = 1

        idx = rng.choice(n_total, size=n_sample, replace=False)

        X_out.append(X_c[idx])
        y_out.append(y_c[idx])

    return np.concatenate(X_out, axis=0), np.concatenate(y_out, axis=0)


def classwise_uncertainty_sampling_by_fraction(X_pool, y_pool, model, fraction=0.2, random_state=42):
    X_selected = []
    y_selected = []
    selected_indices = []

    classes = np.unique(y_pool)
    probs = model.predict_proba(X_pool)
    rng = np.random.RandomState(seed=random_state)

    for c in classes:
        idx_c = np.where(y_pool == c)[0]
        X_c = X_pool[idx_c]
        probs_c = probs[idx_c]


        n_c = len(idx_c)
        n_select = max(1, int(n_c * fraction))

        if n_c == 0:
            continue

        uncertainties = entropy(probs_c.T)  # shape: (n_samples_in_class,)
        sorted_idx = np.argsort(uncertainties)[-n_select:]  # top-k uncertain

        top_k_idx = idx_c[sorted_idx]

        selected_indices.extend(top_k_idx)
        X_selected.append(X_pool[top_k_idx])
        y_selected.append(y_pool[top_k_idx])

    selected_indices = np.array(selected_indices)
    X_selected = np.concatenate(X_selected, axis=0)
    y_selected = np.concatenate(y_selected, axis=0)

    remaining_idx = np.setdiff1d(np.arange(len(X_pool)), selected_indices)
    X_rest = X_pool[remaining_idx]
    y_rest = y_pool[remaining_idx]

    return X_rest, X_selected, y_rest, y_selected

X_pool = x_test_adv_0_1 
y_pool = y_label_0_1
X_train_init, y_train_init = stratified_random_sample(X_pool, y_pool, fraction=0.1, random_state=42)

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_init, y_train_init)



_, X_20, _, y_20 = classwise_uncertainty_sampling_by_fraction(
    x_test_adv_0_1, y_label_0_1, clf, fraction=0.1
)



In [6]:
X_20.shape

(214873, 42)

In [7]:
from xgboost import XGBClassifier

print('xgb')
xgb20 = XGBClassifier()
xgb20.fit(X_20, y_20)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf20 = RandomForestClassifier()
rf20.fit(X_20, y_20)

# print("DT")
# from sklearn.tree import DecisionTreeClassifier
# dt20 = DecisionTreeClassifier()
# dt20.fit(X_20, y_20)

xgb
RF


RandomForestClassifier()

In [8]:
y_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

In [9]:
import numpy as np



base_path_template = "/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = xgb20.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning/WUSTL_Input20/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning/WUSTL_Input20/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[3 6 8] [     4 224487  14262]
(4, 42) (4,)
Save 3 to baseline_Def3.npy
(224487, 42) (224487,)
Save 6 to baseline_Def6.npy
(14262, 42) (14262,)
Save 8 to baseline_Def8.npy
Execution Time: 2.169066 seconds
(716259,)
[ 1  3  4  6  8 10] [419108 271606     37  24148    798    562]
(419108, 42) (419108,)
Save 1 to BIM_Def1.npy
(271606, 42) (271606,)
Save 3 to BIM_Def3.npy
(37, 42) (37,)
Save 4 to BIM_Def4.npy
(24148, 42) (24148,)
Save 6 to BIM_Def6.npy
(798, 42) (798,)
Save 8 to BIM_Def8.npy
(562, 42) (562,)
Save 10 to BIM_Def10.npy
Execution Time: 5.409757 seconds
(716259,)
[ 1  3  4  6  8 10] [450837 242762     82  17066   1272   4240]
(450837, 42) (450837,)
Save 1 to FGSM_Def1.npy
(242762, 42) (242762,)
Save 3 to FGSM_Def3.npy
(82, 42) (82,)
Save 4 to FGSM_Def4.npy
(17066, 42) (17066,)
Save 6 to FGSM_Def6.npy
(1272, 42) (1272,)
Save 8 to FGSM_Def8.npy
(4240, 42) (4240,)
Save 10 to FGSM_Def10.npy
Execution Time: 4.533832 seconds
(716259,)
[ 1  3  4  6  8 10] [419108 271606     

In [10]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = rf20.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning/WUSTL_Input20/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning/WUSTL_Input20/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[3 6 8] [     7 225049  13697]
(7, 42) (7,)
Save 3 to baseline_Def3.npy
(225049, 42) (225049,)
Save 6 to baseline_Def6.npy
(13697, 42) (13697,)
Save 8 to baseline_Def8.npy
Execution Time: 31.591450 seconds
(716259,)
[1 3 6 8] [226609 486106    331   3213]
(226609, 42) (226609,)
Save 1 to BIM_Def1.npy
(486106, 42) (486106,)
Save 3 to BIM_Def3.npy
(331, 42) (331,)
Save 6 to BIM_Def6.npy
(3213, 42) (3213,)
Save 8 to BIM_Def8.npy
Execution Time: 34.766058 seconds
(716259,)
[ 1  3  6  8 10] [442015 252500  18674   3037     33]
(442015, 42) (442015,)
Save 1 to FGSM_Def1.npy
(252500, 42) (252500,)
Save 3 to FGSM_Def3.npy
(18674, 42) (18674,)
Save 6 to FGSM_Def6.npy
(3037, 42) (3037,)
Save 8 to FGSM_Def8.npy
(33, 42) (33,)
Save 10 to FGSM_Def10.npy
Execution Time: 24.345299 seconds
(716259,)
[1 3 6 8] [226609 486106    331   3213]
(226609, 42) (226609,)
Save 1 to PGD_Def1.npy
(486106, 42) (486106,)
Save 3 to PGD_Def3.npy
(331, 42) (331,)
Save 6 to PGD_Def6.npy
(3213, 42) (3213,)
Save

In [11]:
# from sklearn.model_selection import train_test_split

# for attack_name, epsilons in attack_models:
#     start_time = time.time()
#     all_adv_examples = []
#     all_y = []
    
#     if attack_name != "baseline":
#         for epsilon in epsilons:
#             filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
#             x_test_adv = np.load(filename)
#             all_adv_examples.append(x_test_adv)
#             all_y.append(y_test)
    
#         X = np.concatenate(all_adv_examples, axis=0)
#         y = np.concatenate(all_y, axis=0)
#     else:
#         X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
#         y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

#     y_pred = dt20.predict(X)
#     print(y_pred.shape)
#     y_pred = label_encoder.inverse_transform(y_pred)
#     unique_classes, count = np.unique(y_pred, return_counts=True)
#     print(unique_classes, count)
#     for class_num in unique_classes:
#         indices = np.where(y_pred == class_num)[0]
#         X_class = X[indices]
#         y_class = y[indices]
#         print(X_class.shape, y_class.shape)
    
#         x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_ActiveLearning/WUSTL_Input20/x_test_adv_{attack_name}_Def{class_num}.npy"
#         y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_ActiveLearning/WUSTL_Input20/y_test_adv_{attack_name}_Def{class_num}.npy"
        
#         np.save(x_filename, X_class)
#         np.save(y_filename, y_class)
    
#         print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

#     end_time = time.time()
#     result = end_time - start_time
#     print(f"Execution Time: {result:.6f} seconds")
    

In [12]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}20")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/{m_name}_ActiveLearning/WUSTL_Input20/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [13]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input20_ActiveLearning.csv")

In [14]:
all_attack_list

0       1   2       3      4     5       6   7       8    9     10  11  \
0   XGB20       0   0       4      0     0  224487   0   14262    0     0   0   
1   XGB20  419108   0  271606     37     0   24148   0     798    0   562   0   
2   XGB20  450837   0  242762     82     0   17066   0    1272    0  4240   0   
3   XGB20  419108   0  271606     37     0   24148   0     798    0   562   0   
4   XGB20     101   0   17325      0   142  688985   0    2705    0  7001   0   
5   XGB20  104415   0  404979     30     1  203151   0    3086    0   597   0   
6   XGB20       0   0      13      0     0  673456   0   42790    0     0   0   
7   XGB20   60840   0   18897  16833  1394   63732   0  554182  248   133   0   
8   XGB20   23730   0  686904    101     0    5524   0       0    0     0   0   
9   XGB20  429221   0  249934    175     0   34953   0      51    0  1925   0   
10   RF20       0   0       7      0     0  225049   0   13697    0     0   0   
11   RF20  226609   0  486106      0     0     331   0    3213    0     0   0   
12   RF20  442015   0  252500      0     0   18674   0    3037    0    33   0   
13   RF20  226609   0  486106      0     0     331   0    3213    0     0   0   
14   RF20      98   0   20273      0   137  690045   0    2079    0  3627   0   
15   RF20  152341   0  558387      0     0    5372   0     159    0     0   0   
16   RF20       0   0      23      0     0  675140   0   41096    0     0   0   
17   RF20   37669   0   11053  14808  1031   58578   0  592949   88    83   0   
18   RF20    1751   0  712947      0     0    1561   0       0    0     0   0   
19   RF20  421688   0  264607      0     0   25486   0    4478    0     0   0   
20   DT20       0   0       0      0     0       0   0       0    0     0   0   
21   DT20       0   0       0      0     0       0   0       0    0     0   0   
22   DT20       0   0       0      0     0       0   0       0    0     0   0   
23   DT20       0   0       0      0     0       0   0       0    0     0   0   
24   DT20       0   0       0      0     0       0   0       0    0     0   0   
25   DT20       0   0       0      0     0       0   0       0    0     0   0   
26   DT20       0   0       0      0     0       0   0       0    0     0   0   
27   DT20       0   0       0      0     0       0   0       0    0     0   0   
28   DT20       0   0       0      0     0       0   0       0    0     0   0   
29   DT20       0   0       0      0     0       0   0       0    0     0   0   

        12  
0   238753  
1   716259  
2   716259  
3   716259  
4   716259  
5   716259  
6   716259  
7   716259  
8   716259  
9   716259  
10  238753  
11  716259  
12  716259  
13  716259  
14  716259  
15  716259  
16  716259  
17  716259  
18  716259  
19  716259  
20       0  
21       0  
22       0  
23       0  
24       0  
25       0  
26       0  
27       0  
28       0  
29       0